In [ ]:
import mytools
from aip import AipOcr  #导入AipOcr模块，用于做文字识别
import time #时间模块
import requests #用于HTTP请求

In [ ]:
import os
if not os.path.exists('./结果/'):
            os.makedirs('./结果/')

In [ ]:
APP_ID = '25246190'
API_KEY = 'Vwo9GIFTdSqu7cFqgGDbVnWj'
SECRET_KEY = 'HqSfMVTBA8QflAcZdGwyrnwbAe6xGLy7'
client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

In [ ]:
picture = 'D:/advertising/区域/闫寨/竣工资料/拍照图片/1IMG_20211130_212720_1.jpg'

In [ ]:
pic = open(picture,'rb') #以二进制方式打开图片
img = pic.read() #读取
table = client.tableRecognitionAsync(img)    #调用表格识别模块
request_id = table['result'][0]['request_id']

In [ ]:
#判断识别是否完成，直到完成才根据请求ID获取Excel下载路径
result = client.getTableRecognitionResult(request_id)  #通过ID获取识别结果

In [ ]:
result

In [ ]:
while result['result']['ret_msg'] != '已完成': #如果状态是“已完成”，才能获取下载地址
    a = 
    time.sleep(5) #暂停2秒再刷新
    result = client.getTableRecognitionResult(request_id) #持续刷新，直到满足条件

In [ ]:
download_path = result['result']['result_data']

In [ ]:
#下载并将Excel文件名设为图片名
excel_name = picture.split("./结果/")[0] + ".xls" #让excel文件的名字与图片相同
excel = requests.get(download_path) #抓取下载链接
file = open(excel_name, 'wb') #新建excel文件
file.write(excel.content) #写入excel文件并保存
file.close()
print('1')

In [ ]:
f = mytools.othern.file_name_paths('D:/advertising/区域/闫寨/竣工资料/拍照图片/')
#提交识别请求，并储存所有请求ID
for picture in f:
    pic = open(picture,'rb') #以二进制方式打开图片
    img = pic.read() #读取
    table = client.tableRecognitionAsync(img)    #调用表格识别模块
    request_id = table['result'][0]['request_id']
    
    #判断识别是否完成，直到完成才根据请求ID获取Excel下载路径
    result = client.getTableRecognitionResult(request_id)  #通过ID获取识别结果
    a = 1 
    while result['result']['ret_msg'] != '已完成': #如果状态是“已完成”，才能获取下载地址
        time.sleep(2) #暂停2秒再刷新
        result = client.getTableRecognitionResult(request_id) #持续刷新，直到满足条件
        a = a + 1
        if a ==5:
            break
        
    download_path = result['result']['result_data']
    
    #下载并将Excel文件名设为图片名
    excel_name = picture.split("./结果/")[0] + ".xls" #让excel文件的名字与图片相同
    excel = requests.get(download_path) #抓取下载链接
    file = open(excel_name, 'wb') #新建excel文件
    file.write(excel.content) #写入excel文件并保存
    file.close()
    print('1')
mytools.dog()

In [ ]:
# -*- coding: utf-8 -*-
# from PIL import Image
# import pytesseract
##导入通用包
import numpy as np
import pandas as pd
import os
import json
import re
import base64
# import xlwings as xw
# 导入腾讯AI api
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.ocr.v20181119 import ocr_client, models

In [ ]:
# 定义函数
def excelFromPictures(picture, SecretId, SecretKey):
    resp = None
    # try:
    with open(picture, "rb") as f:
        img_data = f.read()
    img_base64 = base64.b64encode(img_data)
    cred = credential.Credential(SecretId, SecretKey)  # ID和Secret从腾讯云申请
    httpProfile = HttpProfile()
    httpProfile.endpoint = "ocr.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = ocr_client.OcrClient(cred, "ap-shanghai", clientProfile)

    req = models.TableOCRRequest()
    params = '{"ImageBase64":"' + str(img_base64) + '"}'
    # params = '{"ImageBase64":"' + str(img_base64, 'utf-8') + '"}'
    req.from_json_string(params)
    resp = client.TableOCR(req)
    #     print(resp.to_json_string())

    # except TencentCloudSDKException as err:
    #     print(err)

    ##提取识别出的数据，并且生成json
    result1 = json.loads(resp.to_json_string())

    rowIndex = []
    colIndex = []
    content = []

    for item in result1['TextDetections']:
        rowIndex.append(item['RowTl'])
        colIndex.append(item['ColTl'])
        content.append(item['Text'])

    ##导出Excel
    ##ExcelWriter方案
    rowIndex = pd.Series(rowIndex)
    colIndex = pd.Series(colIndex)

    index = rowIndex.unique()
    index.sort()

    columns = colIndex.unique()
    columns.sort()

    data = pd.DataFrame(index=index, columns=columns)
    for i in range(len(rowIndex)):
        data.loc[rowIndex[i], colIndex[i]] = re.sub(" ", "", content[i])

    writer = pd.ExcelWriter("../tables/" + re.match(".*\.", f.name).group() + "xlsx", engine='xlsxwriter')
    data.to_excel(writer, sheet_name='Sheet1', index=False, header=False)
    writer.save()

    # xlwings方案
    # wb = xw.Book()
    # sht = wb.sheets('Sheet1')
    # for i in range(len(rowIndex)):
    #     sht[rowIndex[i],colIndex[i]].value = re.sub(" ",'',content[i])
    # wb.save("../tables/" + re.match(".*\.",f.name).group() + "xlsx")
    # wb.close()


# if not ('tables') in os.listdir():
#     os.mkdir("./tables/")

os.chdir("D:/22/")
pictures = os.listdir('.')
for pic in pictures:
    excelFromPictures(pic, "AKID3uVpthEUV1MTKvFH2WlkSt47wWquhDTU", "nJZYHlyvc4QZUAakeIgpiMlvpTrzsVrx")
    print("已经完成" + pic + "的提取.")


In [3]:
# from PIL import Image
# import pytesseract
##导入通用包
import numpy as np
import mytools
import pandas as pd
import os
import json
import re
import base64
import xlwings as xw
##导入腾讯AI api
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.ocr.v20181119 import ocr_client, models
 
#定义函数
def excelFromPictures(picture,SecretId,SecretKey):
    print(picture)
    try:
        with open(picture,"rb") as f:
                img_data = f.read()
        img_base64 = base64.b64encode(img_data)
        cred = credential.Credential(SecretId, SecretKey)  #ID和Secret从腾讯云申请
        httpProfile = HttpProfile()
        httpProfile.endpoint = "ocr.tencentcloudapi.com"
 
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        client = ocr_client.OcrClient(cred, "ap-shanghai", clientProfile)
 
        req = models.TableOCRRequest()
        params = '{"ImageBase64":"' + str(img_base64, 'utf-8') + '"}'
        req.from_json_string(params)
        resp = client.TableOCR(req)
        #     print(resp.to_json_string())
 
    except TencentCloudSDKException as err:
        print(err)
 
    ##提取识别出的数据，并且生成json
    result1 = json.loads(resp.to_json_string())
 
    rowIndex = []
    colIndex = []
    content = []
 
    for item in result1['TextDetections']:
        rowIndex.append(item['RowTl'])
        colIndex.append(item['ColTl'])
        content.append(item['Text'])
 
    ##导出Excel
    ##ExcelWriter方案
    rowIndex = pd.Series(rowIndex)
    colIndex = pd.Series(colIndex)
 
    index = rowIndex.unique()
    index.sort()
 
    columns = colIndex.unique()
    columns.sort()
 
    data = pd.DataFrame(index = index, columns = columns)
    for i in range(len(rowIndex)):
        data.loc[rowIndex[i],colIndex[i]] = re.sub(" ","",content[i])
 
    writer = pd.ExcelWriter(re.match(".*\.",f.name).group() + "xlsx", engine='xlsxwriter')
    data.to_excel(writer,sheet_name = 'Sheet1', index=False,header = False)
    writer.save()
 
    #xlwings方案  
    # wb = xw.Book()
    # sht = wb.sheets('Sheet1')
    # for i in range(len(rowIndex)):
    #     sht[rowIndex[i],colIndex[i]].value = re.sub(" ",'',content[i])
    # wb.save("../tables/" + re.match(".*\.",f.name).group() + "xlsx")
    # wb.close()
f = mytools.othern.file_name_paths('D:/advertising/区域/闫寨/竣工资料/拍照图片/')
for pic in f:
    excelFromPictures(pic,"AKID3uVpthEUV1MTKvFH2WlkSt47wWquhDTU", "nJZYHlyvc4QZUAakeIgpiMlvpTrzsVrx")
    print("已经完成" + pic + "的提取.")

D:/advertising/区域/闫寨/竣工资料/拍照图片/1IMG_20211130_212720_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/1IMG_20211130_212720_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212627_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212627_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212659_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212659_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212808_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_212808_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213026_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213026_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213053_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213053_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213107_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213107_1.jpg的提取.
D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213138_1.jpg
已经完成D:/advertising/区域/闫寨/竣工资料/拍照图片/IMG_20211130_213138_1.jpg的提取.
D:/advertising/区域/闫寨/竣